# News data collection

In [1]:
import os
import sys

# Go up two directories to get the project root
project_root = os.path.dirname(os.path.dirname(os.getcwd()))

# Add the project root to sys.path
sys.path.append(project_root)

from src.utils import data_source




In [2]:
import logging

# Initialize logging
logging.basicConfig(filename='debug.log', level=logging.DEBUG)

ticker = 'FOREX:USD'
initial_date = '20230918T2243'
sort = 'EARLIEST'
file_path = None

data_collector = data_source.FinancialAnalytics(file_path=file_path, ticker=ticker, date_from=initial_date, sort=sort)
last_date = initial_date
all_data = []

iteration_count = 0  # To keep track of the number of iterations

while True:
    logging.debug(f"Starting iteration {iteration_count}")
    print(f"Starting iteration {iteration_count}")

    # Fetch data from the specified date
    data_collector.date_from = last_date
    sentiment_data = data_collector.fetch_news_sentiment_data()

    # Log and print the type and content of sentiment_data
    logging.debug(f"Type of sentiment_data: {type(sentiment_data)}")
    logging.debug(f"Content of sentiment_data: {sentiment_data.head()}")
    print(f"Type of sentiment_data: {type(sentiment_data)}")
    print(f"Content of sentiment_data: {sentiment_data.head()}")

    # If no data is retrieved, break
    if sentiment_data.empty:
        logging.debug("Breaking loop, sentiment_data is empty.")
        print("Breaking loop, sentiment_data is empty.")
        break

    # Append the fetched data to the all_data list
    all_data.append(sentiment_data)

    # Update the last_date based on the last timestamp of the fetched data
    sentiment_data['time_published(est)'] = sentiment_data['time_published(est)'].str.replace('-', '').str.replace(' ', 'T').str.replace(':', '')
    last_date = sentiment_data['time_published(est)'].iloc[-1]
    logging.debug(f"Updated last_date to: {last_date}")
    print(f"Updated last_date to: {last_date}")

    iteration_count += 1

# Combine all fetched data
combined_data = pd.concat(all_data, ignore_index=True)
combined_data = combined_data.drop_duplicates()

logging.debug("Data collection completed.")
print("Data collection completed.")


Starting iteration 0
Type of sentiment_data: <class 'pandas.core.frame.DataFrame'>
Content of sentiment_data:       ticker relevance_score ticker_sentiment_score ticker_sentiment_label  \
0  FOREX:USD        0.149966               0.018743                Neutral   
1  FOREX:USD         0.06515               0.024153                Neutral   
2  FOREX:USD        0.246404               0.173937       Somewhat-Bullish   
3  FOREX:USD        0.075397               0.041496                Neutral   
4  FOREX:USD        0.269776               0.044056                Neutral   

  time_published(est)  
0    2023-09-18 19:16  
1    2023-09-18 20:28  
2    2023-09-18 20:30  
3    2023-09-18 22:00  
4    2023-09-18 22:26  
Updated last_date to: 20231004T0820
Starting iteration 1
Type of sentiment_data: <class 'pandas.core.frame.DataFrame'>
Content of sentiment_data:       ticker relevance_score ticker_sentiment_score ticker_sentiment_label  \
0  FOREX:USD        0.324139               0.121448  

ValueError: If using all scalar values, you must pass an index

In [111]:
combined_data = pd.concat(all_data, ignore_index=True)
combined_data = combined_data.drop_duplicates()
combined_data.to_csv(r'C:\Users\jessi\OneDrive\1 Projects\1-svts\data\external\news-analysis\usd_sentiment7.csv')

In [117]:
import pandas as pd
import os

# Directory containing the CSV files
directory = r"C:\Users\jessi\OneDrive\1 Projects\1-svts\data\external\news-analysis"

# List of CSV filenames
files = [
    os.path.join(directory, "usd_sentiment.csv"),
    os.path.join(directory, "usd_sentiment1.csv"),
    os.path.join(directory, "usd_sentiment2.csv"),
    os.path.join(directory, "usd_sentiment3.csv"),
    os.path.join(directory, "usd_sentiment4.csv"),
    os.path.join(directory, "usd_sentiment5.csv"),
    os.path.join(directory, "usd_sentiment6.csv"),
    os.path.join(directory, "usd_sentiment7.csv")
]

# Read and combine all files into one DataFrame
all_data = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

# Drop duplicates
all_data.drop_duplicates(inplace=True)

# Define your target folder

# Ensure the folder exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Get the timestamp of the last data entry
last_timestamp = all_data['time_published(est)'].iloc[-1]

# Format the timestamp for filename 
formatted_timestamp = last_timestamp.replace(' ', '_').replace(':', '-').replace('/', '-')

# Construct the new filename with path included
new_filename = os.path.join(directory, f"usd_sentiment_combined_{formatted_timestamp}.csv")

# Save the combined data to the new filename
all_data.to_csv(new_filename, index=False)


# MT5 data collection 1 minute data